In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import datetime
import os.path
import backtrader as bt
from backtrader.indicators import SMA, ROC100
import numpy as np
from pprint import pformat
import pandas as pd
import matplotlib.pyplot as plt
import datetime


## Strategy

In [ ]:
# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('rp1', 10), ('rp2', 15), ('rp3', 20), ('rp4', 30),
        ('rma1', 10), ('rma2', 10), ('rma3', 10), ('rma4', 10),
        ('rsignal', 9), 
        ('rfactors', [1.0, 2.0, 3.0, 4.0]),
        ('_rmovav', SMA),
        ('_smovav', SMA),
        ('printlog', False),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add the KST indicator
        rcma1 = self.p._rmovav(ROC100(period=self.p.rp1), period=self.p.rma1)
        rcma2 = self.p._rmovav(ROC100(period=self.p.rp2), period=self.p.rma2)
        rcma3 = self.p._rmovav(ROC100(period=self.p.rp3), period=self.p.rma3)
        rcma4 = self.p._rmovav(ROC100(period=self.p.rp4), period=self.p.rma4)
        self.kst = sum([rfi * rci for rfi, rci in zip(self.params.rfactors, [rcma1, rcma2, rcma3, rcma4])])

        # Add the Signal
        self.signal = self.params._smovav(self.kst, period=self.params.rsignal)
        
    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.signal > 0:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.signal < 0 :
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('Signal for this Period:%2d, Ending KST Signal: %2d, %.2f' %
                 (self.params.rsignal, self.signal[0], self.broker.getvalue()), doprint=True)

## Analyzers

In [ ]:
class DecisiveAnalyzer(bt.Analyzer):
    
    def __init__(self):
        self.kstper = self.strategy.params.rsignal
        self.equity = []
        self.equitydf = pd.DataFrame()

    def start(self):
        # Not needed ... but could be used
        self.init_cash = self.strategy.broker.cash 
        self.num_trades = 0
        self.first_trade_open = None

    def next(self):
        # Not needed ... but could be used
        pass

    def notify_trade(self, trade):
        if not self.first_trade_open:
            self.first_trade_open = self.strategy.datetime.datetime()
            self.equitydf.at[self.first_trade_open, 'equity'] = self.init_cash
        if trade.isclosed:
            self.num_trades += 1
            self.equitydf.at[self.strategy.datetime.datetime(), 'equity'] = self.strategy.broker.getvalue()
            self.equity.append(self.strategy.broker.getvalue())
    def stop(self):
        self.final_cash = self.strategy.broker.cash
        self.final_val = self.strategy.broker.get_value()

    def get_analysis(self):
        # Add trade profit to dataframe
        self.equitydf['net_profit'] = self.equitydf['equity'].diff()

        outp = {
            'params': (self.kstper),
            'profit': self.final_val - self.init_cash,
            'num_trades': self.num_trades,
            'equitydf': self.equitydf,
            'trades' : self.equity
        }
        
        return outp
    

In [ ]:
def best_result_from_cerebro_opti_run(result):
    params  = []
    n_trades = []
    profit   = []
    trades   = []
    for res in result:
        r = res[0].analyzers.decisive.get_analysis()
        params.append(r['params'])
        n_trades.append(r['num_trades'])
        profit.append(r['profit'])
        trades.append(r['trades'])

    prof_ind = np.argmax(profit) 
    best_params = params[prof_ind]
    best_profit = profit[prof_ind]
    best_ntrades = n_trades[prof_ind]
    best_trades = trades[prof_ind]
    
    print('best:{} profit:{} trades:{}'.format(best_params, best_profit, best_ntrades))
    return (best_params, best_profit, best_ntrades, best_trades)

In [ ]:
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()
df['2017-01-01':'2017-04-01']['close'].plot()

## In-sample Parameters

In [ ]:
periods = [
{'run': 0, 'oos': ('2018-01-01', '2018-04-01'), 'is': ('2017-01-01', '2018-01-01')},
{'run': 1, 'oos': ('2018-02-01', '2018-05-01'), 'is': ('2017-02-01', '2018-02-01')},
{'run': 2, 'oos': ('2018-03-01', '2018-06-01'), 'is': ('2017-03-01', '2018-03-01')},
{'run': 3, 'oos': ('2018-04-01', '2018-07-01'), 'is': ('2017-04-01', '2018-04-01')},
{'run': 4, 'oos': ('2018-05-01', '2018-08-01'), 'is': ('2017-05-01', '2018-05-01')},
{'run': 5, 'oos': ('2018-06-01', '2018-09-01'), 'is': ('2017-06-01', '2018-06-01')},
{'run': 6, 'oos': ('2018-07-01', '2018-10-01'), 'is': ('2017-07-01', '2018-07-01')},
{'run': 7, 'oos': ('2018-08-01', '2018-11-01'), 'is': ('2017-08-01', '2018-08-01')},
{'run': 8, 'oos': ('2018-09-01', '2018-12-01'), 'is': ('2017-09-01', '2018-09-01')},
{'run': 9, 'oos': ('2018-10-01', '2019-01-01'), 'is': ('2017-10-01', '2018-10-01')},
{'run': 10, 'oos': ('2018-11-01', '2019-02-01'), 'is': ('2017-11-01', '2018-11-01')},
{'run': 11, 'oos': ('2018-12-01', '2019-03-01'), 'is': ('2017-12-01', '2018-12-01')},
{'run': 12, 'oos': ('2019-01-01', '2019-04-01'), 'is': ('2018-01-01', '2019-01-01')},

]

In [ ]:

best_oos_params = []
for oos_is in periods:
    start_date, end_date = oos_is['is']
    print('Start/End: {} - {}'.format(start_date, end_date))


    cerebro = bt.Cerebro()

    strats = cerebro.optstrategy(
        TestStrategy,
        rsignal=(5, 9, 15) )

    cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

    cerebro.optreturn = False
    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    results = cerebro.run(maxcpus=1)
    best_param, best_profit, best_trades, best_equity = best_result_from_cerebro_opti_run(results)

    best_oos_params.append({'train_param': best_param, 
                                      'train_profit': best_profit, 
                                      'train_numtrades': best_trades, 
                                      'train_tradeslist': best_equity, 
                                      'train_period': oos_is['is'],
                                      'test_period': oos_is['oos'],
                                     })

print('The best OOS parameters are: {}'.format(pformat(best_oos_params)))




## Out-of-sample Parameters

In [ ]:
# Get in-sample and out-of-sample best parameters with correct pre-allocation
runnbr = 0
test_netprofit = []
for bestoos in best_oos_params:
    # Get the insample time period
    start_date, end_date = bestoos['test_period']
    
    print('Start/End: {} - {}'.format(start_date, end_date))
    cerebro = bt.Cerebro()

    cerebro.addstrategy(TestStrategy,
        rsignal=bestoos['train_param'])

    
    cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

    cerebro.broker.setcash(100000.0)
    cerebro.addsizer(bt.sizers.FixedSize, stake=1000)
    cerebro.broker.setcommission(commission=0.0)

    data = bt.feeds.PandasData(dataname = df[start_date:end_date])
    cerebro.adddata(data)

    results = cerebro.run(maxcpus=1)
    r = results[0].analyzers.decisive.get_analysis()
    best_oos_params[runnbr]['test_numtrades'] = r['num_trades']
    best_oos_params[runnbr]['test_tradeslist'] = r['trades']
    best_oos_params[runnbr]['test_profit'] = r['profit']
    test_netprofit.append(r['profit'])
    runnbr += 1
    
#     #Put the best parameters in a list
# period_netprofit = []
# for result in results:
#         r = result.analyzers.decisive.get_analysis()
#         df = r['equitydf']
#         kstperiod = r['params']
#         period_profit = df[start_date:insample_end]['net_profit'].sum()
#         print('tkstperiod {}: {}'.format(kstperiod, period_profit))
#         period_netprofit.append((kstperiod, period_profit))

#     # Get the best profit parameters
# max_set = max(test_netprofit)
# max_profit = max_set

# # Add it to the dictionary
# print('\t\tBest is: {} profit'.format(max_profit))

# Walkforward Efficiency

In [ ]:
def days_from_date_tuple(mytuple):
    insample_days_start = datetime.datetime.strptime(mytuple[0], '%Y-%m-%d')
    insample_days_end = datetime.datetime.strptime(mytuple[1], '%Y-%m-%d')
    insample_days = (insample_days_end - insample_days_start).days
    return insample_days

oos_trades = []
wfe_list = []
for values in best_oos_params:
    oos_trades_list = values['test_tradeslist']
    oos_trades.extend(oos_trades_list)
    
    insample_days = days_from_date_tuple(values['train_period'])
    oos_days = days_from_date_tuple(values['test_period'])

    insample_annual_profit = 365/insample_days * values['train_profit']
    oos_annual_profit = 365/oos_days * values['test_profit']

    walkforward_efficiency_pct = round(oos_annual_profit * 100 / insample_annual_profit, 1)
    
    wfe_list.append(walkforward_efficiency_pct)

    print('Run {} WFE:{}'.format(runnbr, walkforward_efficiency_pct))
    print('\t IS_DAYS: {} OOS_DAYS: {} IS_PROFIT: {:0.2f} OOS_PROFIT: {:0.2f}'.format(insample_days, oos_days,
                                                                            insample_annual_profit, oos_annual_profit))

print('Average WFE: {}%'.format(np.asarray(wfe_list).mean()))

## Walkforward Equity Curve

In [ ]:
# Create a Stratey
class WalkforwardStrategy(bt.Strategy):

    params = (
        ('rp1', 10), ('rp2', 15), ('rp3', 20), ('rp4', 30),
        ('rma1', 10), ('rma2', 10), ('rma3', 10), ('rma4', 10),
        ('rsignal', 9), 
        ('rfactors', [1.0, 2.0, 3.0, 4.0]),
        ('_rmovav', SMA),
        ('_smovav', SMA),
        ('printlog', False),
        ('live', False),
        ('walkforward', None),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
              # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add the KST indicator
        rcma1 = self.p._rmovav(ROC100(period=self.p.rp1), period=self.p.rma1)
        rcma2 = self.p._rmovav(ROC100(period=self.p.rp2), period=self.p.rma2)
        rcma3 = self.p._rmovav(ROC100(period=self.p.rp3), period=self.p.rma3)
        rcma4 = self.p._rmovav(ROC100(period=self.p.rp4), period=self.p.rma4)
        self.kst = sum([rfi * rci for rfi, rci in zip(self.params.rfactors, [rcma1, rcma2, rcma3, rcma4])])

        # Add the Signal
        self.signal = self.params._smovav(self.kst, period=self.params.rsignal)
        
        # Trim the indicators if we are running live, just need the current one
        if not self.params.live:
            pass
        
        
        self.current_row = None
        self.wfkst = []
        
        if self.params.walkforward:
            for run in self.params.walkforward:
                self.wfkst.append({
                    'kst': self.signal,
                    'test_period': run['test_period'],
                    'train_param': run['train_param'],
                }) 

        print(pformat(self.wfkst))
        
        

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):

        # Walk-forward logic
        for row in self.wfkst:
            start, end = row['test_period']
            period_start = datetime.datetime.strptime(start, '%Y-%m-%d')
            period_end = datetime.datetime.strptime(end, '%Y-%m-%d')
            if self.datetime.datetime() >= period_start and self.datetime.datetime() < period_end:
                self.current_row = row
        
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.signal > 0:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.signal < 0:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()

    def stop(self):
        self.log('Signal for this Period:%2d, Ending KST Signal: %2d, %.2f' %
                 (self.params.rsignal, self.signal[0], self.broker.getvalue()), doprint=True)

In [ ]:
# Create a cerebro entity
cerebro = bt.Cerebro()

# Add a strategy
cerebro.addstrategy(WalkforwardStrategy, 
                    walkforward=best_oos_params,
                    live=False)

# Load data
fname_symbol = 'CL'
folder_name = '5min'
suffix = '5min_20160103_20190405'

df = pd.read_parquet(os.path.join('../data/processed/{}/'.format(folder_name), '{}_{}.parquet'.format(fname_symbol, suffix)))
df = (df.resample('4h', label='left', base=18).agg({'Open': 'first', 'High': 'max', 'Low': 'min', 'Close': 'last', 'Volume': 'sum'}))
df.columns = [col_name.lower() for col_name in df.columns]
df = df.dropna()

# periods
start_date = best_oos_params[0]['test_period'][0]
end_date = best_oos_params[-1]['test_period'][0]

print('Start: {} End: {}'.format(start_date, end_date))
data = bt.feeds.PandasData(dataname = df[start_date:end_date])


# Add the Data Feed to Cerebro
cerebro.adddata(data)
cerebro.addanalyzer(DecisiveAnalyzer, _name='decisive')

# Set our desired cash start
cerebro.broker.setcash(100000.0)
cerebro.addsizer(bt.sizers.FixedSize, stake=1000)

# Print out the starting conditions
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

# Run over everything
results = cerebro.run()

In [ ]:
# Print out the final result
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
r = results[0].analyzers.decisive.get_analysis()
cumsum = r['equitydf']['net_profit'].cumsum()

plt.plot(cumsum)
plt.title('Walkforward Equity Curve')
plt.xlabel('Trades')
plt.ylabel('Equity')
plt.show()

cerebro.plot(volume=False, iplot=False)

In [ ]:
avg_wfe = np.round(np.asarray(wfe_list).mean(),1)
print('Average WFE: {}%'.format(avg_wfe))